In [1]:
import splitfolders

In [2]:
splitfolders.ratio('Images', output="Dataset", seed=1337, ratio=(.8, 0.1,0.1))

Copying files: 1084 files [00:01, 620.72 files/s]


In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [4]:
IMAGE_SIZE=[224,224]
train_path="Dataset\train"
valid_path="Dataset\val"

In [5]:
vgg=VGG16(input_shape=IMAGE_SIZE+[3],weights="imagenet",include_top=False)

In [6]:
for layer in vgg.layers:
    layer.trainable = False

In [7]:
folders = glob('Dataset/train/*')

In [8]:
len(folders)

2

In [9]:
x = Flatten()(vgg.output)

In [10]:
prediction = Dense(len(folders), activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
training_set = train_datagen.flow_from_directory('Dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size =4,
                                                 class_mode = 'categorical')

Found 866 images belonging to 2 classes.


In [15]:
test_set = test_datagen.flow_from_directory('Dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 4,
                                            class_mode = 'categorical')

Found 111 images belonging to 2 classes.


In [16]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=30,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\akrit\AppData\Local\Temp/ipykernel_21780/1525006258.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


217/217 [==============================] - 234s 1s/step - loss: 0.1592 - accuracy: 0.9908 - val_loss: 0.4560 - val_accuracy: 0.9820


In [18]:
import tensorflow as tf

In [19]:
tf.keras.models.save_model(model,"mymodel2.hdf5")

In [20]:
%%writefile app.py
import os
import streamlit as st
import tensorflow as tf
st.set_option('deprecation.showfileUploaderEncoding',False)
def load_model():
    model=tf.keras.models.load_model("mymodel2.hdf5")
    return model
model=load_model()
st.write("Logo detection")
file=st.file_uploader("Please upload image",type=["jpg","png","jpeg"])
import cv2
from PIL import Image,ImageOps
import numpy as np
def import_and_predict(image_data,model):
    size=(224,224)
    image=ImageOps.fit(image_data,size,Image.ANTIALIAS)
    image=np.asarray(image)
    image_reshape=image[np.newaxis,...]
    prediction=model.predict(image_reshape)
if file is None:
    st.text("Please upload an image file")
else:
    image=Image.open(file)
    st.image(image,use_column_width=True)
    predictions=import_and_predict(image,model)
    class_names=['Real','Fake']
    strings="The detected logo in this image is : "+class_names[np.argmax(predictions)]
    st.success(strings)    

Overwriting app.py
